## Project Structure

```arduino
ticket-reporter/
│
├── app/
│   ├── __init__.py
│   ├── models.py
│   ├── views.py
│   ├── forms.py
│   ├── data/
│   │   └── sqlite/
│   │       └── app.db
│   ├── templates/
│   │   ├── base.html
│   │   ├── home.html
│   │   ├── task.html
│   │   ├── ticket.html
│   │   ├── project.html
│   │   └── about.html
│   └── static/
│       ├── css/
│       │   └── styles.css
│       ├── js/
│       │   └── scripts.js
│       └── favicon.ico
├── db_init.py
├── run.py
├── config.py
├── runner.ipynb
└── requirements.txt
```

## Verify Test Data

In [ ]:
import random
import string

import pandas as pd

from app import create_app, db
from app.models import Task, Ticket, Project

####################################################

# 通过 __init__.py 中的 create_app() 函数创建app
app = create_app()
# sqlite:///d:\\Projects\\Py-Projects\\ticket-reporter\\data\\sqlite\\app.db
db_path = app.config['SQLALCHEMY_DATABASE_URI']
print(db_path)

with app.app_context():

    # 读取表格内容为 list，其中的元素都为各自对应模型的实例
    tasks_list = Task.query.all()
    tickets_list = Ticket.query.all()
    projects_list = Project.query.all()

    # 读取表格内容为 DataFrame
    tasks_df = pd.read_sql(Task.query.statement, db.session.bind)
    tickets_df = pd.read_sql(Ticket.query.statement, db.session.bind)
    projects_df = pd.read_sql(Project.query.statement, db.session.bind)


In [ ]:
import sqlite3
import os
import pandas as pd
from app import create_app, db
from app.models import Task, Ticket, Project

# 创建 Flask 应用实例
app = create_app()

# 提取数据库路径
db_uri = app.config['SQLALCHEMY_DATABASE_URI']
if db_uri.startswith('sqlite:///'):
    db_path = db_uri[len('sqlite:///'):]
elif db_uri.startswith('sqlite://'):
    db_path = db_uri[len('sqlite://'):]
else:
    raise ValueError("Unsupported database URI scheme")

# 在 Jupyter Notebook 中手动指定路径
db_path = os.path.join(os.getcwd(), 'app', 'data', 'sqlite', 'report.db')

# 使用 SQLAlchemy 的连接引擎
from sqlalchemy import create_engine
engine = create_engine(f'sqlite:///{db_path}')

# 使用 pandas 读取 SQL 数据
with app.app_context():
    tasks_df = pd.read_sql(Task.query.statement, db.session.bind)
    tickets_df = pd.read_sql(Ticket.query.statement, db.session.bind)
    projects_df = pd.read_sql(Project.query.statement, db.session.bind)

# 显示 DataFrame 内容
print(tasks_df.head())
print(tickets_df.head())
print(projects_df.head())

In [ ]:
#random_ticket = random.choice(tickets)
#vars(random_ticket)

for tick in tickets_list:
    print(
        f"{tick.id}|{tick.title}|{tick.report_on}|{tick.description}|{tick.project_id}|{tick.update_on}"
    )

In [ ]:
import random
import json
import os
import matplotlib.pyplot as plt

from shapely.geometry import shape, Point

# 
def load_europe_polygon(geojson_path):
    with open(geojson_path, 'r') as file:
        geojson = json.load(file)
    return shape(geojson['features'][0]['geometry'])

def generate_random_point_in_polygon(polygon):
    min_x, min_y, max_x, max_y = polygon.bounds
    while True:
        random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        if polygon.contains(random_point):
            return random_point
        
europe_json = os.path.join(os.getcwd(), 'data', 'json', 'europe.json')
# 加载欧洲的多边形边界
europe_polygon = load_europe_polygon(europe_json)